In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import numpy as np
import pymysql
import sqlalchemy as alch
from getpass import getpass
import os
import re

# Exploring and preprocessing 5 datasets

## I. First dataset: Extinct languages 

In [2]:
extinct_languages_df = pd.read_csv("/Users/usuari/Desktop/Ironhack/BOOTCAMP/projects/project-4-sql-tableau/data/languages.csv")

The full detailed dataset includes names of languages, number of speakers, the names of countries where the language is still spoken, and the degree of endangerment. The UNESCO endangerment classification is as follows:

- Vulnerable: most children speak the language, but it may be restricted to certain domains (e.g., home)

- Definitely endangered: children no longer learn the language as a 'mother tongue' in the home

- Severely endangered: language is spoken by grandparents and older generations; while the parent generation may understand it, they do not speak it to children or among themselves

- Critically endangered: the youngest speakers are grandparents and older, and they speak the language partially and infrequently

- Extinct: there are no speakers left

In [5]:
extinct_languages_df.shape

(2722, 15)

In [4]:
extinct_languages_df.sample(2)

,ID,Name in English,Name in French,Name in Spanish,Countries,Country codes alpha 3,ISO639-3 codes,Degree of endangerment,Alternate names,Name in the language,Number of speakers,Sources,Latitude,Longitude,Description of the location
1254,2489,Kumak,kumak,kumak,New Caledonia (France),NCL,nee,Vulnerable,"nêlêmwa, nixumwak",NaN,847.0,http://lacito.vjf.cnrs.fr/,-20.1278,164.0259,"North of the ""Grande Terre"" (Poum, Koumak and ..."
976,386,Lule Saami,lule saami,saamí de Lule,"Norway, Sweden","NOR, SWE",smj,Severely endangered,NaN,julevsábme,2000.0,Pekka Sammallahti: The Saami languages: an int...,67.2720,17.7978,Jokkmokk County and parts of Gällivare and oth...


In [7]:
extinct_languages_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2722 entries, 0 to 2721
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           2722 non-null   int64  
 1   Name in English              2722 non-null   object 
 2   Name in French               2699 non-null   object 
 3   Name in Spanish              2701 non-null   object 
 4   Countries                    2721 non-null   object 
 5   Country codes alpha 3        2721 non-null   object 
 6   ISO639-3 codes               2458 non-null   object 
 7   Degree of endangerment       2722 non-null   object 
 8   Alternate names              1583 non-null   object 
 9   Name in the language         27 non-null     object 
 10  Number of speakers           2539 non-null   float64
 11  Sources                      2079 non-null   object 
 12  Latitude                     2719 non-null   float64
 13  Longitude         

In [7]:
extinct_languages_df.dtypes

ID                               int64
Name in English                 object
Name in French                  object
Name in Spanish                 object
Countries                       object
Country codes alpha 3           object
ISO639-3 codes                  object
Degree of endangerment          object
Alternate names                 object
Name in the language            object
Number of speakers             float64
Sources                         object
Latitude                       float64
Longitude                      float64
Description of the location     object
dtype: object

In [8]:
extinct_languages_df.columns

Index(['ID', 'Name in English', 'Name in French', 'Name in Spanish',
       'Countries', 'Country codes alpha 3', 'ISO639-3 codes',
       'Degree of endangerment', 'Alternate names', 'Name in the language',
       'Number of speakers', 'Sources', 'Latitude', 'Longitude',
       'Description of the location'],
      dtype='object')

In [9]:
desired_columns = ['ID', 'Name in English', 'Name in Spanish', 'Countries', 'Country codes alpha 3', 'Degree of endangerment', 'Number of speakers', 'Latitude', 'Longitude']

extinct_languages = extinct_languages_df[desired_columns]
extinct_languages.shape

(2722, 9)

In [10]:
extinct_languages.columns = [i.lower().replace(" ", "_") for i in extinct_languages.columns]
extinct_languages.sample(2)

,id,name_in_english,name_in_spanish,countries,country_codes_alpha_3,degree_of_endangerment,number_of_speakers,latitude,longitude
129,389,Gagauz (Bessarabia),gagauso (Besarabia),"Republic of Moldova, Ukraine","MDA, UKR",Definitely endangered,180000.0,46.0922,28.7951
1778,1735,Molo,molo,Sudan,SDN,Critically endangered,100.0,10.8980,34.2773


In [11]:
extinct_languages.rename(columns={"country_codes_alpha_3": "country_code"}, inplace=True)
extinct_languages.head(3)

/var/folders/_b/4s7pp99j1qq9lxsqpjphc78m0000gn/T/ipykernel_71267/233109667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extinct_languages.rename(columns={"country_codes_alpha_3": "country_code"}, inplace=True)


,id,name_in_english,name_in_spanish,countries,country_code,degree_of_endangerment,number_of_speakers,latitude,longitude
0,1022,South Italian,napolitano-calabrés,Italy,ITA,Vulnerable,7500000.0,40.9798,15.2490
1,1023,Sicilian,siciliano,Italy,ITA,Vulnerable,5000000.0,37.4399,14.5019
2,383,Low Saxon,bajo sajón,"Germany, Denmark, Netherlands, Poland, Russian...","DEU, DNK, NLD, POL, RUS",Vulnerable,4800000.0,53.4029,10.3601


In [12]:
# I check how many null values are there

extinct_languages.isna().sum()

id                          0
name_in_english             0
name_in_spanish            21
countries                   1
country_code                1
degree_of_endangerment      0
number_of_speakers        183
latitude                    3
longitude                   3
dtype: int64

In [13]:
# I check if there's duplicated data.

print(f"Dataset has {extinct_languages.duplicated().sum()} duplicated data.")

Dataset has 0 duplicated data.


In [13]:
# I save my cleaned dataframe.

extinct_languages.to_csv("extinct_languages.csv", index=False)

folder_path = "/Users/usuari/Desktop/Ironhack/BOOTCAMP/projects/project-4-sql-tableau/data"
file_name = "extinct_languages.csv"

# Combine the folder path and filename to create the full file path
full_file_path = f"{folder_path}/{file_name}"

# Export the DataFrame to the specified folder
extinct_languages.to_csv(full_file_path, index=False)

## II. Second dataset: List of languages by total number of speakers (most spoken languages in the world). 

In [14]:
top_languages = pd.read_csv("/Users/usuari/Desktop/Ironhack/BOOTCAMP/projects/project-4-sql-tableau/data/list_languages.csv")

In [15]:
top_languages.shape

(45, 7)

In [16]:
top_languages.dtypes

Unnamed: 0                       int64
Language                        object
Family                          object
Branch                          object
First-language(L1) speakers     object
Second-language(L2) speakers    object
Total speakers(L1+L2)           object
dtype: object

In [17]:
top_languages.isna().sum()

Unnamed: 0                      0
Language                        0
Family                          0
Branch                          0
First-language(L1) speakers     0
Second-language(L2) speakers    0
Total speakers(L1+L2)           0
dtype: int64

In [18]:
print(f"Dataset has {top_languages.duplicated().sum()} duplicated data.")

Dataset has 0 duplicated data.


In [19]:
top_languages.columns = [i.lower().replace(" ", "_") for i in top_languages.columns]

In [21]:
top_languages.rename(columns={
    "first-language(l1)_speakers": "first_speakers",
    "second-language(l2)_speakers": "second_speakers",
    "total_speakers(l1+l2)": "total_speakers"
}, inplace=True)
top_languages.head(4)

,unnamed:_0,language,family,branch,first_speakers,second_speakers,total_speakers
0,0,English(excl. creole languages),Indo-European,Germanic,372.9 million,1.080 billion[5],1.452 billion
1,1,"Mandarin Chinese(incl. Standard Chinese, but e...",Sino-Tibetan,Sinitic,929.0 million,198.7 million[6],1.118 billion
2,2,Hindi(excl. Urdu),Indo-European,Indo-Aryan,343.9 million,258.3 million[7],602.2 million
3,3,Spanish,Indo-European,Romance,474.7 million,73.6 million[8],548.3 million


In [27]:
# I create a function to convert the values of the last three columns from objects to floats.

def to_float(string):
    try:
        # Extract the numerical part
        num_str = re.search(r'\d+\.\d+', string).group()
        
        # Check for 'million' or 'billion' and multiply accordingly
        if 'million' in string:
            return float(num_str) * 1e6
        elif 'billion' in string:
            return float(num_str) * 1e9
        else:
            return float(num_str)
    except (ValueError, AttributeError):
        # Handle cases where conversion is not possible
        return None

In [28]:
top_languages['first_speakers'] = top_languages['first_speakers'].apply(to_float)

In [30]:
top_languages['second_speakers'] = top_languages['second_speakers'].apply(to_float)
top_languages['total_speakers'] = top_languages['total_speakers'].apply(to_float)
top_languages.head(5)

,unnamed:_0,language,family,branch,first_speakers,second_speakers,total_speakers
0,0,English(excl. creole languages),Indo-European,Germanic,372900000.0,1.080000e+09,1.452000e+09
1,1,"Mandarin Chinese(incl. Standard Chinese, but e...",Sino-Tibetan,Sinitic,929000000.0,1.987000e+08,1.118000e+09
2,2,Hindi(excl. Urdu),Indo-European,Indo-Aryan,343900000.0,2.583000e+08,6.022000e+08
3,3,Spanish,Indo-European,Romance,474700000.0,7.360000e+07,5.483000e+08
4,4,French,Indo-European,Romance,79900000.0,1.942000e+08,2.741000e+08


In [31]:
# I save my cleaned dataframe.

top_languages.to_csv("top_languages.csv", index=False)

folder_path = "/Users/usuari/Desktop/Ironhack/BOOTCAMP/projects/project-4-sql-tableau/data"
file_name = "top_languages.csv"

# Combine the folder path and filename to create the full file path
full_file_path = f"{folder_path}/{file_name}"

# Export the DataFrame to the specified folder
top_languages.to_csv(full_file_path, index=False)

## III. Third dataset: All countries details

In [2]:
details_df = pd.read_csv("/Users/usuari/Desktop/Ironhack/BOOTCAMP/projects/project-4-sql-tableau/data/detailed_countries.csv")

In [3]:
details_df.shape

(194, 64)

In [4]:
details_df.columns

Index(['country', 'country_long', 'currency', 'capital_city', 'region',
       'continent', 'demonym', 'latitude', 'longitude', 'agricultural_land',
       'forest_area', 'land_area', 'rural_land', 'urban_land',
       'central_government_debt_pct_gdp', 'expense_pct_gdp', 'gdp',
       'inflation', 'self_employed_pct', 'tax_revenue_pct_gdp',
       'unemployment_pct', 'vulnerable_employment_pct',
       'electricity_access_pct', 'alternative_nuclear_energy_pct',
       'electricty_production_coal_pct',
       'electricty_production_hydroelectric_pct',
       'electricty_production_gas_pct', 'electricty_production_nuclear_pct',
       'electricty_production_oil_pct', 'electricty_production_renewable_pct',
       'energy_imports_pct', 'fossil_energy_consumption_pct',
       'renewable_energy_consumption_pct', 'co2_emissions',
       'methane_emissions', 'nitrous_oxide_emissions',
       'greenhouse_other_emissions', 'urban_population_under_5m',
       'health_expenditure_pct_gdp', 'healt

In [5]:
desired_columns = ['country', 'country_long', 'region', 'continent', 'latitude', 'longitude', 'rural_land', 'urban_land', 'population', 'rural_population', 'urban_population', 'democracy_score', 'democracy_type']

details = details_df[desired_columns]
details.shape

(194, 13)

In [6]:
details.dtypes

country              object
country_long         object
region               object
continent            object
latitude            float64
longitude           float64
rural_land          float64
urban_land          float64
population            int64
rural_population      int64
urban_population      int64
democracy_score     float64
democracy_type       object
dtype: object

In [7]:
details.isna().sum()

country             0
country_long        0
region              0
continent           0
latitude            0
longitude           0
rural_land          0
urban_land          0
population          0
rural_population    0
urban_population    0
democracy_score     0
democracy_type      0
dtype: int64

In [8]:
print(f"Dataset has {details.duplicated().sum()} duplicated data.")

Dataset has 0 duplicated data.


In [9]:
details.sample(3)

,country,country_long,region,continent,latitude,longitude,rural_land,urban_land,population,rural_population,urban_population,democracy_score,democracy_type
57,Ethiopia,Federal Democratic Republic of Ethiopia,Eastern Africa,Africa,8.000000,38.000000,1124620.0,5760.660,123379924,95420799,27959125,3.35,Authoritarian
171,The Gambia,Republic of The Gambia,Western Africa,Africa,13.466667,-16.566667,10197.8,338.749,2705992,978162,1727830,4.31,Hybrid regime
69,Guinea-Bissau,Republic of Guinea-Bissau,Western Africa,Africa,12.000000,-15.000000,33293.4,196.264,2105566,1157198,948368,1.98,Authoritarian


In [10]:
# I save my cleaned dataframe.

details.to_csv("details.csv", index=False)

folder_path = "/Users/usuari/Desktop/Ironhack/BOOTCAMP/projects/project-4-sql-tableau/data"
file_name = "details.csv"

# Combine the folder path and filename to create the full file path
full_file_path = f"{folder_path}/{file_name}"

# Export the DataFrame to the specified folder
details.to_csv(full_file_path, index=False)

## IV. Fourth dataset: World Countries and Official Languages

In [11]:
official = pd.read_csv("/Users/usuari/Desktop/Ironhack/BOOTCAMP/projects/project-4-sql-tableau/data/official_languages.csv")

In [12]:
official.shape

(198, 2)

In [13]:
official.dtypes

Country             object
Languages Spoken    object
dtype: object

In [14]:
official.sample(4)

,Country,Languages Spoken
95,Latvia,"Latvian 58% (official), Russian 38%, Lithuania..."
58,Fiji,"English (official), Fijian, Hindustani"
45,Czech Republic,Czech
121,Myanmar,"Burmese, minority languages"


In [15]:
official.columns = [i.lower().replace(" ", "_") for i in official]
official.head(3)

,country,languages_spoken
0,Afghanistan,"Dari Persian, Pashtu (both official), other Tu..."
1,Albania,"Albanian (Tosk is the official dialect), Greek"
2,Algeria,"Arabic (official), French, Berber dialects"


In [16]:
official.isna().sum()

country             0
languages_spoken    0
dtype: int64

In [17]:
print(f"Dataset has {official.duplicated().sum()} duplicated data.")

Dataset has 0 duplicated data.


### I'm going to join the last two dataframes:

In [19]:
country_lan = pd.merge(details, official, on='country', how='left')  

In [20]:
country_lan.shape

(194, 14)

In [21]:
country_lan.sample(3)

,country,country_long,region,continent,latitude,longitude,rural_land,urban_land,population,rural_population,urban_population,democracy_score,democracy_type,languages_spoken
189,Vietnam,Socialist Republic of Vietnam,South-Eastern Asia,Asia,16.166667,107.833333,297627.000,28615.4000,98186856,60123739,38063117,3.08,Authoritarian,Vietnamese (official); English (increasingly f...
174,Tonga,Kingdom of Tonga,Polynesia,Oceania,-20.000000,-175.000000,701.786,29.6473,106858,82147,24711,0.00,Unknown,"Tongan (an Austronesian language), English"
185,Uruguay,Oriental Republic of Uruguay,South America,Americas,-33.000000,-56.000000,173707.000,1017.8700,3422794,147591,3275203,8.38,Full democracy,"Spanish, Portunol, or Brazilero"


In [58]:
# I'm going to create a function, so that I can add a new column with the number of spoken languages for each country.


def count_languages(language_string):
    if pd.isna(language_string):
        return 0  # Return 0 for NaN values
    
    excluded_terms = ["many", "bilingual"]

    # Split the string by commas, semicolons, and "and", but exclude content inside parentheses
    separators = re.compile(r',|;|and|\([^)]*\)')
    languages = [lang.strip() for lang in separators.split(language_string) if lang]

    # Exclude specified terms
    languages = [lang for lang in languages if all(term not in lang.lower() for term in excluded_terms)]

    return len(languages)

In [59]:
country_lan['count_languages'] = country_lan['languages_spoken'].apply(count_languages)

In [63]:
country_lan.sample(3)

,country,country_long,region,continent,latitude,longitude,rural_land,urban_land,population,rural_population,urban_population,democracy_score,democracy_type,languages_spoken,count_languages
29,Cameroon,Republic of Cameroon,Middle Africa,Africa,6.000000,12.0,463837.0,2352.11,27914536,11519492,16395044,3.28,Authoritarian,"French, English (both official); 24 major Afri...",3
141,Rwanda,Republic of Rwanda,Eastern Africa,Africa,-2.000000,30.0,22337.3,1506.23,13776698,11335329,2441369,3.35,Authoritarian,"Kinyarwanda, French, and English (all official...",6
40,Croatia,Republic of Croatia,Southern Europe,Europe,45.166667,15.5,54777.1,1648.41,3854000,1610240,2243760,6.57,Flawed democracy,"Croatian 96% (official), other 4% (including I...",2


In [64]:
country_lan.isna().sum()

country              0
country_long         0
region               0
continent            0
latitude             0
longitude            0
rural_land           0
urban_land           0
population           0
rural_population     0
urban_population     0
democracy_score      0
democracy_type       0
languages_spoken    16
count_languages      0
dtype: int64

In [65]:
# I save the new dataframe.

country_lan.to_csv("country_lan.csv", index=False)

folder_path = "/Users/usuari/Desktop/Ironhack/BOOTCAMP/projects/project-4-sql-tableau/data"
file_name = "country_lan.csv"

# Combine the folder path and filename to create the full file path
full_file_path = f"{folder_path}/{file_name}"

# Export the DataFrame to the specified folder
country_lan.to_csv(full_file_path, index=False)